<a href="https://colab.research.google.com/github/vvrgit/AI/blob/main/LoadForecasting_LR_DT_RF_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras  #Keras is the deep learning library that helps you to code Deep Neural Networks with fewer lines of code
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adadelta,SGD,Adagrad,Adam
#import pylab as plt
#import seaborn as sns #For data visualization
import pandas as pd # For Data manipulation

In [ ]:
load_data=pd.read_excel("/content/drive/MyDrive/Load.xlsx")
load_data.head()
min_load=load_data.min()
print(min_load)
max_load=load_data.max()
print(max_load)

L(T-1)     1229.18
L(T-2)     1229.18
L(T-24)    1229.18
L(T-48)    1229.18
P(T-1)      598.75
P(T-2)      598.75
P(T-24)     598.75
P(T-48)     598.75
Day           0.00
Season        0.00
L(T)       1229.18
dtype: float64
L(T-1)     15096.73
L(T-2)     15096.73
L(T-24)    15096.73
L(T-48)    15096.73
P(T-1)     20000.00
P(T-2)     20000.00
P(T-24)    20000.00
P(T-48)    20000.00
Day            1.00
Season         2.00
L(T)       15096.73
dtype: float64


In [ ]:
load_data.head()

,L(T-1),L(T-2),L(T-24),L(T-48),P(T-1),P(T-2),P(T-24),P(T-48),Day,Season,L(T)
0,4851.90,4309.93,5257.03,4574.50,2313.77,2333.89,2328.34,2285.62,1,2,5604.93
1,5604.93,4851.90,5527.90,5217.88,2005.82,2313.77,2143.30,2097.21,1,2,6012.75
2,6012.75,5604.93,5820.83,5314.53,1886.45,2005.82,2024.52,2008.19,1,2,6163.53
3,6163.53,6012.75,5910.98,5471.73,1810.42,1886.45,1942.67,2008.32,1,2,6310.30
4,6310.30,6163.53,6189.18,5974.50,1805.42,1810.42,2071.41,2038.73,1,2,6606.13


In [ ]:
print(load_data.shape) # details about number of samples and features
load_data.describe()

(23304, 11)


,L(T-1),L(T-2),L(T-24),L(T-48),P(T-1),P(T-2),P(T-24),P(T-48),Day,Season,L(T)
count,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000
mean,6599.603310,6599.702593,6603.358441,6605.967243,5059.075687,5058.746726,5053.714292,5048.181380,0.143194,0.988671,6599.482203
std,2012.373466,2012.203412,2011.188930,2010.564244,3135.934965,3135.818725,3133.813298,3131.785046,0.350278,0.789283,2012.563564
min,1229.180000,1229.180000,1229.180000,1229.180000,598.750000,598.750000,598.750000,598.750000,0.000000,0.000000,1229.180000
25%,5134.032500,5134.032500,5137.087500,5139.557500,2995.015000,2994.965000,2994.637500,2993.872500,0.000000,0.000000,5134.032500
50%,6355.980000,6355.980000,6359.210000,6362.490000,3879.070000,3878.760000,3875.995000,3872.970000,0.000000,1.000000,6355.980000
75%,7883.382500,7883.382500,7888.057500,7890.882500,6000.290000,6000.282500,5999.212500,5990.867500,0.000000,2.000000,7883.382500
max,15096.730000,15096.730000,15096.730000,15096.730000,20000.000000,20000.000000,20000.000000,20000.000000,1.000000,2.000000,15096.730000


In [ ]:
from sklearn.preprocessing import MinMaxScaler
ms=MinMaxScaler()
load_data=ms.fit_transform(load_data)
print(load_data)

[[0.2612372  0.22215532 0.29045145 ... 1.         1.         0.31553879]
 [0.31553879 0.2612372  0.3099841  ... 1.         1.         0.34494702]
 [0.34494702 0.31553879 0.33110751 ... 1.         1.         0.35581988]
 ...
 [0.04409575 0.04811016 0.09046587 ... 0.         0.         0.043971  ]
 [0.043971   0.04409575 0.08935392 ... 0.         0.         0.05531402]
 [0.05531402 0.043971   0.08745741 ... 0.         0.         0.05772108]]


In [ ]:
dataset=pd.DataFrame(data=load_data[0:,0:])
print(dataset[1].values)

[0.22215532 0.2612372  0.31553879 ... 0.04811016 0.04409575 0.043971  ]


In [ ]:
X=dataset.iloc[:,0:10].values
Y=dataset.iloc[:,10:].values
print(X)
print(Y)

[[0.2612372  0.22215532 0.29045145 ... 0.08694646 1.         1.        ]
 [0.31553879 0.2612372  0.3099841  ... 0.07723523 1.         1.        ]
 [0.34494702 0.31553879 0.33110751 ... 0.07264687 1.         1.        ]
 ...
 [0.04409575 0.04811016 0.09046587 ... 0.48456929 0.         0.        ]
 [0.043971   0.04409575 0.08935392 ... 0.48456929 0.         0.        ]
 [0.05531402 0.043971   0.08745741 ... 0.48456929 0.         0.        ]]
[[0.31553879]
 [0.34494702]
 [0.35581988]
 ...
 [0.043971  ]
 [0.05531402]
 [0.05772108]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print(y_test[0:5])
print(X_test.shape)
print(X_train.shape)

[[0.36760278]
 [0.32372193]
 [0.42424942]
 [0.27153679]
 [0.39462053]]
(4661, 10)
(18643, 10)


In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Create and train the SVM model
svm_model = SVR(kernel='rbf')  # You can choose different kernels like 'linear', 'poly', or 'rbf'
svm_model.fit(X_train, y_train.ravel())  # We use ravel to convert y_train to a 1D array

# Predict on the test set
svm_predictions = svm_model.predict(X_test)

# Calculate and print test loss
svm_test_loss = mean_squared_error(y_test, svm_predictions)
print('SVM Test Loss:', svm_test_loss)

# Calculate and print training loss (if needed)
svm_train_predictions = svm_model.predict(X_train)
svm_train_loss = mean_squared_error(y_train, svm_train_predictions)
print('SVM Training Loss:', svm_train_loss)

# Calculate and print R-squared (Coefficient of determination)
r2 = r2_score(y_test, svm_predictions)
print('SVM R-squared:', r2)

SVM Test Loss: 0.001955865497109175
SVM Training Loss: 0.0018700251707086414
SVM R-squared: 0.9103685304750746


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)  # You can adjust the number of estimators
rf_model.fit(X_train, y_train.ravel())

# Predict on the test set
rf_predictions = rf_model.predict(X_test)

# Calculate and print test loss
rf_test_loss = mean_squared_error(y_test, rf_predictions)
print('Random Forest Test Loss:', rf_test_loss)

# Calculate and print training loss (if needed)
rf_train_predictions = rf_model.predict(X_train)
rf_train_loss = mean_squared_error(y_train, rf_train_predictions)
print('Random Forest Training Loss:', rf_train_loss)


Random Forest Test Loss: 0.0017489793817471258
Random Forest Training Loss: 0.0002493277312064685


In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate MSE for the SVM model
svm_test_mse = mean_squared_error(y_test, svm_predictions)
print('SVM Test MSE:', svm_test_mse)

# Calculate MSE for the Random Forest model
rf_test_mse = mean_squared_error(y_test, rf_predictions)
print('Random Forest Test MSE:', rf_test_mse)

SVM Test MSE: 0.001955865497109175
Random Forest Test MSE: 0.0017489793817471258


In [ ]:
import numpy as np

# Calculate RMSE for the SVM model
svm_test_rmse = np.sqrt(svm_test_mse)
print('SVM Test RMSE:', svm_test_rmse)

# Calculate RMSE for the Random Forest model
rf_test_rmse = np.sqrt(rf_test_mse)
print('Random Forest Test RMSE:', rf_test_rmse)

SVM Test RMSE: 0.044225168141106885
Random Forest Test RMSE: 0.04182080082622911


In [ ]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=None)
regressor.fit(X_train, y_train)
regressor.get_depth()

37

In [ ]:
y_pred_test =  regressor.predict(X_test)
y_pred_train =  regressor.predict(X_train)
print(y_pred_test)
print(y_pred_train)

[0.51769058 0.34486265 0.44147308 ... 0.33097627 0.40769855 0.68641685]
[0.60331926 0.48791027 0.1732094  ... 0.59904021 0.35952277 0.52554344]


In [ ]:
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train))
print('Root mean square error', rmse)
mse = (mean_squared_error(y_train, y_pred_train))
print('Mean square error', mse)
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.0
Mean square error 0.0
Mean absolute error 0.0


In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create and train the Decision Tree model
dt_model = DecisionTreeRegressor(random_state=0)
dt_model.fit(X_train, y_train)

# Predict on the test set
dt_predictions = dt_model.predict(X_test)

# Calculate and print test loss
dt_test_loss = mean_squared_error(y_test, dt_predictions)
print('Decision Tree Test Loss:', dt_test_loss)

# Calculate and print training loss (if needed)
dt_train_predictions = dt_model.predict(X_train)
dt_train_loss = mean_squared_error(y_train, dt_train_predictions)
print('Decision Tree Training Loss:', dt_train_loss)

# Calculate and print R-squared (Coefficient of determination)
r2 = r2_score(y_test, dt_predictions)
print('Decision Tree R-squared:', r2)

Decision Tree Test Loss: 0.0034483152134961183
Decision Tree Training Loss: 0.0
Decision Tree R-squared: 0.8419740210011173


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Create and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict on the test set
lr_predictions = lr_model.predict(X_test)

# Calculate and print test loss (MSE)
lr_test_loss = mean_squared_error(y_test, lr_predictions)
print('Linear Regression Test Loss (MSE):', lr_test_loss)

# Calculate and print training loss (MSE)
lr_train_predictions = lr_model.predict(X_train)
lr_train_loss = mean_squared_error(y_train, lr_train_predictions)
print('Linear Regression Training Loss (MSE):', lr_train_loss)

# Calculate and print R-squared (Coefficient of determination)
r2 = r2_score(y_test, lr_predictions)
print('Linear Regression R-squared:', r2)

Linear Regression Test Loss (MSE): 0.001856109304822003
Linear Regression Training Loss (MSE): 0.0018528345686538012
Linear Regression R-squared: 0.9149400585899301
